In [108]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.metrics import SCORERS

In [3]:
df_features = pd.read_csv('features.csv')
df_features_test = pd.read_csv('features_test.csv')

In [200]:
df_features_test.shape

(17177, 103)

In [49]:
df_features.shape

(97230, 109)

# Подготовка данных для бустинга

### Преобразование каждого столбца с видом героя в подмасив, где в шапке виды героев, а в ячейке  
### "0" или "1" есть герой такого типа или нет

In [4]:
r1_hero = pd.get_dummies(df_features['r1_hero'])
r1_hero.columns = ['r1_hero_' + str(name) for name in r1_hero.columns ]
r2_hero = pd.get_dummies(df_features['r2_hero'])
r2_hero.columns = ['r2_hero_' + str(name) for name in r2_hero.columns ]
r3_hero = pd.get_dummies(df_features['r3_hero'])
r3_hero.columns = ['r3_hero_' + str(name) for name in r3_hero.columns ]
r4_hero = pd.get_dummies(df_features['r4_hero'])
r4_hero.columns = ['r4_hero_' + str(name) for name in r4_hero.columns ]
r5_hero = pd.get_dummies(df_features['r5_hero'])
r5_hero.columns = ['r5_hero_' + str(name) for name in r5_hero.columns ]
d1_hero = pd.get_dummies(df_features['d1_hero'])
d1_hero.columns = ['d1_hero_' + str(name) for name in d1_hero.columns ]
d2_hero = pd.get_dummies(df_features['d2_hero'])
d2_hero.columns = ['d2_hero_' + str(name) for name in d2_hero.columns ]
d3_hero = pd.get_dummies(df_features['d3_hero'])
d3_hero.columns = ['d3_hero_' + str(name) for name in d3_hero.columns ]
d4_hero = pd.get_dummies(df_features['d4_hero'])
d4_hero.columns = ['d4_hero_' + str(name) for name in d4_hero.columns ]
d5_hero = pd.get_dummies(df_features['d5_hero'])
d5_hero.columns = ['d5_hero_' + str(name) for name in d5_hero.columns ]

In [5]:
# для примера, как выглядит результат преобразования выбора первого игрока героя в каждой игре
# r1-5 номер игрока команды r, hero_1-112 id героя, 1 или 0 выбрвн данный герой, этим игроком или нет
r1_hero.head()

,r1_hero_1,r1_hero_2,r1_hero_3,r1_hero_4,r1_hero_5,r1_hero_6,r1_hero_7,r1_hero_8,r1_hero_9,r1_hero_10,...,r1_hero_100,r1_hero_101,r1_hero_102,r1_hero_103,r1_hero_104,r1_hero_105,r1_hero_106,r1_hero_109,r1_hero_110,r1_hero_112
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# приклеиваем новые подмассивы
df_features_classifier = pd.concat([df_features, r1_hero, r2_hero, r3_hero, r4_hero, r5_hero, d1_hero, d2_hero, d3_hero, d4_hero, d5_hero], axis=1).copy()

In [14]:
# удаляем исходные столбцы
df_features_classifier.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                   'd3_hero', 'd4_hero', 'd5_hero'], axis = 1, inplace=True)

In [15]:
# аналогичное преобразование для признака lobby_type
lobby_type = pd.get_dummies(df_features_classifier['lobby_type'])
lobby_type.columns = ['lobby_type_' + str(name) for name in lobby_type.columns ]
df_features_classifier = pd.concat([df_features_classifier, lobby_type], axis=1)
df_features_classifier.drop(['lobby_type'], axis = 1, inplace=True)

In [8]:
missing_data = df_features_classifier.isnull()

In [9]:
# проверка наличия NaN в определенных столбцах
for column in missing_data.columns.values.tolist():
    if True in missing_data[column].value_counts():
        print(column)
        print (missing_data[column].value_counts())
        print("")

first_blood_time
False    77677
True     19553
Name: first_blood_time, dtype: int64

first_blood_team
False    77677
True     19553
Name: first_blood_team, dtype: int64

first_blood_player1
False    77677
True     19553
Name: first_blood_player1, dtype: int64

first_blood_player2
False    53243
True     43987
Name: first_blood_player2, dtype: int64

radiant_bottle_time
False    81539
True     15691
Name: radiant_bottle_time, dtype: int64

radiant_courier_time
False    96538
True       692
Name: radiant_courier_time, dtype: int64

radiant_flying_courier_time
False    69751
True     27479
Name: radiant_flying_courier_time, dtype: int64

radiant_first_ward_time
False    95394
True      1836
Name: radiant_first_ward_time, dtype: int64

dire_bottle_time
False    81087
True     16143
Name: dire_bottle_time, dtype: int64

dire_courier_time
False    96554
True       676
Name: dire_courier_time, dtype: int64

dire_flying_courier_time
False    71132
True     26098
Name: dire_flying_courier_time,

In [10]:
# удаление признаков с результатами игры
df_features_classifier.drop(['match_id', 'start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 
                  'barracks_status_radiant', 'barracks_status_dire'], axis = 1, inplace=True)

In [11]:
# замена NaN
df_features_classifier['first_blood_time'].fillna(500, inplace = True) #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['dire_first_ward_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['dire_flying_courier_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['dire_bottle_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['radiant_first_ward_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['radiant_flying_courier_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['radiant_courier_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['radiant_bottle_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['dire_courier_time'].fillna(500, inplace = True)  #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['first_blood_player2'].fillna(20, inplace = True) # заменяем на номер несуществуещего игрока (всего 10)
df_features_classifier['first_blood_player1'].fillna(20, inplace = True) # заменяем на номер несуществуещего игрока (всего 10)
df_features_classifier['first_blood_time'].fillna(500, inplace = True) #событие не произошло в первые пять/300 минут/секунд (заменяем на 500секун)
df_features_classifier['first_blood_team'].fillna(5, inplace = True) # если событие произошло 0 или 1, если нет заменяем на 5

In [12]:
# контрольная проверка на наличие NaN
missing_data = df_features_classifier.isnull()
for column in missing_data.columns.values.tolist():
    if True in missing_data[column].value_counts():
        print(column)
        print (missing_data[column].value_counts())
        print("")

In [13]:
y = df_features_classifier['radiant_win'].values

In [14]:
X = df_features_classifier.drop('radiant_win', axis = 1).values

In [15]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

In [16]:
train, test = zip(*kf.split(X))

In [19]:
#оценка на кросс-валидации
%%time
results = []

clf = GradientBoostingClassifier(n_estimators = 10)
clf.fit(X,y)
result = cross_val_score(clf, X, y, cv=kf)

In [17]:
# обучение модели на кросс-валидации с метрикой roc_auc_score
%%time
acaccuracy = np.empty([5,1])

for train_index, test_index, i in zip(train,test,range(5)):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    clf = GradientBoostingClassifier(n_estimators = 30).fit(X_train, y_train)
    y_predict_proba = clf.predict_proba(X_test)
    acaccuracy[i] = roc_auc_score(y_test,  y_predict_proba[:, 1])

CPU times: user 20min 37s, sys: 11.9 s, total: 20min 49s
Wall time: 20min 50s


In [18]:
print('roc_auc_score: ', np.mean(acaccuracy))

roc_auc_score:  0.6896764188466291


# Подготовка данных для логистической регрессии

### Преобразование категорийных признаков

In [6]:
# Домнажаем на -1 выбор героев игроками команды d
# Таким образом выбор героя командой r: 1, выбор героя командой d: -1
d1_hero=d1_hero*(-1)
d2_hero=d2_hero*(-1)
d3_hero=d3_hero*(-1)
d4_hero=d4_hero*(-1)
d5_hero=d5_hero*(-1)

In [7]:
# Складываем выборы всех игроков двух команд, чтобы получить единую матрицу, где выбор героя командой r: 1, выбор героя командой d: -1
hero = r1_hero.values+r2_hero.values+r3_hero.values+r4_hero.values+r5_hero.values+d1_hero.values+d2_hero.values+d3_hero.values+d4_hero.values+d5_hero.values

In [8]:
# имена столбцов для нового подмассива выбора игроков, не все id героев существуют, например нет 108, 106
hero_columns = [name for name in pd.get_dummies(df_features['r1_hero']).columns]

In [9]:
hero_df = pd.DataFrame(hero, columns = hero_columns)
hero_df.head()

,1,2,3,4,5,6,7,8,9,10,...,100,101,102,103,104,105,106,109,110,112
0,0,0,0,-1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_features_logreg = df_features.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                   'd3_hero', 'd4_hero', 'd5_hero', 'match_id', 'start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 
                  'barracks_status_radiant', 'barracks_status_dire', 'lobby_type'], axis = 1).copy()

In [16]:
df_features_logreg = pd.concat([df_features_logreg, hero_df, lobby_type], axis=1)

In [17]:
# здесь заполняем NaN нулями, т.к. тогда пропущенные признаки не будут влиять на построение модели логистической регрессии
df_features_logreg['first_blood_time'].fillna(0, inplace = True)
df_features_logreg['dire_first_ward_time'].fillna(0, inplace = True)
df_features_logreg['dire_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg['dire_bottle_time'].fillna(0, inplace = True)
df_features_logreg['radiant_first_ward_time'].fillna(0, inplace = True)
df_features_logreg['radiant_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg['radiant_courier_time'].fillna(0, inplace = True)
df_features_logreg['radiant_bottle_time'].fillna(0, inplace = True)
df_features_logreg['dire_courier_time'].fillna(0, inplace = True)
df_features_logreg['first_blood_player2'].fillna(0, inplace = True)
df_features_logreg['first_blood_player1'].fillna(0, inplace = True)
df_features_logreg['first_blood_time'].fillna(0, inplace = True)
df_features_logreg['first_blood_team'].fillna(0, inplace = True)

In [18]:
y_logreg = df_features_logreg['radiant_win'].values

In [19]:
X_logreg = df_features_logreg.drop('radiant_win', axis = 1).values

In [20]:
X_logreg = normalize(X_logreg)

In [21]:
%%time
# подбор лучшего параметра С для модели logreg
grid = {'C': [0.1, 0.5, 1.5, 2]} # задаем С 
cv = KFold(n_splits=5, shuffle=True, random_state=42) # объект разбивающий тренировочные данные на 5 частей для кросс валидации
logreg = LogisticRegression(solver='lbfgs',max_iter=1000, random_state=42) # непосредственно классификатор
gs = GridSearchCV(logreg, grid, scoring='roc_auc', cv=cv) # объект подбирающий лучшее С для классификатора метрика точность кросс валидация
gs.fit(X_logreg, y_logreg) #тренируем

CPU times: user 8min 37s, sys: 12min 37s, total: 21min 14s
Wall time: 5min 27s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.5, 1.5, 2]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=0)

In [22]:
print('roc_auc_score: ', gs.best_score_)
log_reg = gs.best_estimator_

roc_auc_score:  0.712638676998325


### Удаление категорийных признаков

In [127]:
df_features_logreg_noncategory = df_features.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                   'd3_hero', 'd4_hero', 'd5_hero', 'match_id', 'start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 
                  'barracks_status_radiant', 'barracks_status_dire', 'lobby_type'], axis = 1).copy()

In [134]:
df_features_logreg_noncategory['first_blood_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['dire_first_ward_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['dire_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['dire_bottle_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['radiant_first_ward_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['radiant_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['radiant_courier_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['radiant_bottle_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['dire_courier_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['first_blood_player2'].fillna(0, inplace = True)
df_features_logreg_noncategory['first_blood_player1'].fillna(0, inplace = True)
df_features_logreg_noncategory['first_blood_time'].fillna(0, inplace = True)
df_features_logreg_noncategory['first_blood_team'].fillna(0, inplace = True)

In [141]:
y_noncategory = df_features_logreg_noncategory['radiant_win'].values
X_noncategory = df_features_logreg_noncategory.drop('radiant_win', axis = 1).values

In [143]:
X_noncategory = normalize(X_noncategory)

In [144]:
%%time
# подбор лучшего параметра С для модели logreg
grid = {'C': [0.1, 0.5, 1.5, 2]} # задаем С 
cv = KFold(n_splits=5, shuffle=True, random_state=42) # объект разбивающий тренировочные данные на 5 частей для кросс валидации
logreg = LogisticRegression(solver='lbfgs',max_iter=1000, random_state=42) # непосредственно классификатор
gs = GridSearchCV(logreg, grid, scoring='roc_auc', cv=cv) # объект подбирающий лучшее С для классификатора метрика точность кросс валидация
gs.fit(X_noncategory, y_noncategory) #тренируем

CPU times: user 4min 16s, sys: 7min 24s, total: 11min 41s
Wall time: 2min 59s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.5, 1.5, 2]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=0)

In [138]:
print('roc_auc_score: ', gs.best_score_)
log_reg_noncategory = gs.best_estimator_

roc_auc_score:  0.7129436168463892


### Построение модели со всеми признаками (без преобразования или удаления, категорийных признаков)

In [145]:
df_features_logreg_all = df_features.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                   'd3_hero', 'd4_hero', 'd5_hero', 'match_id', 'start_time', 'lobby_type'], axis = 1).copy()

In [147]:
df_features_logreg_all['first_blood_time'].fillna(0, inplace = True)
df_features_logreg_all['dire_first_ward_time'].fillna(0, inplace = True)
df_features_logreg_all['dire_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg_all['dire_bottle_time'].fillna(0, inplace = True)
df_features_logreg_all['radiant_first_ward_time'].fillna(0, inplace = True)
df_features_logreg_all['radiant_flying_courier_time'].fillna(0, inplace = True)
df_features_logreg_all['radiant_courier_time'].fillna(0, inplace = True)
df_features_logreg_all['radiant_bottle_time'].fillna(0, inplace = True)
df_features_logreg_all['dire_courier_time'].fillna(0, inplace = True)
df_features_logreg_all['first_blood_player2'].fillna(0, inplace = True)
df_features_logreg_all['first_blood_player1'].fillna(0, inplace = True)
df_features_logreg_all['first_blood_time'].fillna(0, inplace = True)
df_features_logreg_all['first_blood_team'].fillna(0, inplace = True)

In [148]:
y_all = df_features_logreg_all['radiant_win'].values
X_all = df_features_logreg_all.drop('radiant_win', axis = 1).values

In [149]:
X_all = normalize(X_all)

In [150]:
%%time
# подбор лучшего параметра С для модели logreg
grid = {'C': [0.1, 0.5, 1.5, 2]} # задаем С 
cv = KFold(n_splits=5, shuffle=True, random_state=42) # объект разбивающий тренировочные данные на 5 частей для кросс валидации
logreg = LogisticRegression(solver='lbfgs',max_iter=1000, random_state=42) # непосредственно классификатор
gs = GridSearchCV(logreg, grid, scoring='roc_auc', cv=cv) # объект подбирающий лучшее С для классификатора метрика точность кросс валидация
gs.fit(X_all, y_all) #тренируем

CPU times: user 2min 33s, sys: 4min 12s, total: 6min 46s
Wall time: 1min 45s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.5, 1.5, 2]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=0)

In [151]:
print('roc_auc_score: ', gs.best_score_)
log_reg_all = gs.best_estimator_

roc_auc_score:  0.995314740844149


### Преобразование тестовой выборки для предсказания

In [23]:
r1_hero = pd.get_dummies(df_features_test['r1_hero'])
r1_hero.columns = ['r1_hero_' + str(name) for name in r1_hero.columns ]
r2_hero = pd.get_dummies(df_features_test['r2_hero'])
r2_hero.columns = ['r2_hero_' + str(name) for name in r2_hero.columns ]
r3_hero = pd.get_dummies(df_features_test['r3_hero'])
r3_hero.columns = ['r3_hero_' + str(name) for name in r3_hero.columns ]
r4_hero = pd.get_dummies(df_features_test['r4_hero'])
r4_hero.columns = ['r4_hero_' + str(name) for name in r4_hero.columns ]
r5_hero = pd.get_dummies(df_features_test['r5_hero'])
r5_hero.columns = ['r5_hero_' + str(name) for name in r5_hero.columns ]
d1_hero = pd.get_dummies(df_features_test['d1_hero'])
d1_hero.columns = ['d1_hero_' + str(name) for name in d1_hero.columns ]
d2_hero = pd.get_dummies(df_features_test['d2_hero'])
d2_hero.columns = ['d2_hero_' + str(name) for name in d2_hero.columns ]
d3_hero = pd.get_dummies(df_features_test['d3_hero'])
d3_hero.columns = ['d3_hero_' + str(name) for name in d3_hero.columns ]
d4_hero = pd.get_dummies(df_features_test['d4_hero'])
d4_hero.columns = ['d4_hero_' + str(name) for name in d4_hero.columns ]
d5_hero = pd.get_dummies(df_features_test['d5_hero'])
d5_hero.columns = ['d5_hero_' + str(name) for name in d5_hero.columns ]
lobby_type = pd.get_dummies(df_features_test['lobby_type'])
lobby_type.columns = ['lobby_type_' + str(name) for name in lobby_type.columns ]

In [24]:
d1_hero.shape

(17177, 108)

In [25]:
d1_hero=d1_hero*(-1)
d2_hero=d2_hero*(-1)
d3_hero=d3_hero*(-1)
d4_hero=d4_hero*(-1)
d5_hero=d5_hero*(-1)

In [26]:
hero = r1_hero.values+r2_hero.values+r3_hero.values+r4_hero.values+r5_hero.values+d1_hero.values+d2_hero.values+d3_hero.values+d4_hero.values+d5_hero.values

In [27]:
hero_columns = [name for name in pd.get_dummies(df_features_test['r1_hero']).columns]
hero_df = pd.DataFrame(hero, columns = hero_columns)

In [28]:
df_features_test = pd.concat([df_features_test, hero_df, lobby_type], axis=1).copy()

In [29]:
df_features_test.drop(['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                   'd3_hero', 'd4_hero', 'd5_hero', 'match_id', 'start_time', 'lobby_type'], axis = 1, inplace=True)

In [31]:
X_test = df_features_test.values
X_test = normalize(X_test)

In [32]:
y_predict = log_reg.predict_proba(X_test)

In [35]:
print('максимальная вероятность предсказания победы составила: ', np.max(y_predict[:, :1]), '\n',
      'минимальная вероятность предсказания победы составила: ', np.min(y_predict[:, :1]))

максимальная вероятность предсказания победы составила:  0.9911774789592208 
 минимальная вероятность предсказания победы составила:  0.02316872622409194
